<a href="https://colab.research.google.com/github/vishwvir-singh/Text-completion-with-GPT-2/blob/main/Text_completion_with_GPT-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/Colab_Notebooks_data/Text-completion-with-GPT-2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab_Notebooks_data/Text-completion-with-GPT-2


In [43]:
#Clone GPT-2
import os
if not os.path.exists('gpt-2'):
   !git clone https://github.com/openai/gpt-2.git
%cd gpt-2/

/gdrive/MyDrive/Colab_Notebooks_data/Text-completion-with-GPT-2/gpt-2


In [23]:
%ll ./

total 35
-rw------- 1 root   551 Jun  5 18:13 CONTRIBUTORS.md
-rw------- 1 root  2188 Jun  5 18:13 DEVELOPERS.md
-rw------- 1 root   279 Jun  5 18:13 Dockerfile.cpu
-rw------- 1 root   548 Jun  5 18:13 Dockerfile.gpu
-rw------- 1 root 14754 Jun  5 18:13 domains.txt
-rw------- 1 root  1075 Jun  5 18:13 download_model.py
-rw------- 1 root  1403 Jun  5 18:13 LICENSE
-rw------- 1 root  4989 Jun  5 18:13 model_card.md
-rw------- 1 root  2827 Jun  5 18:13 README.md
-rw------- 1 root    58 Jun  5 18:13 requirements.txt
drwx------ 2 root  4096 Jun  5 18:13 src/


In [37]:
%ll src

total 21
-rw------- 1 root 4242 Jun  5 18:13 encoder.py
-rw------- 1 root 2870 Jun  5 18:13 generate_unconditional_samples.py
-rw------- 1 root 3412 Jun  5 18:13 interactive_conditional_samples.py
-rw------- 1 root 6503 Jun  5 18:13 model.py
-rw------- 1 root 3166 Jun  5 18:13 sample.py


In [25]:
!pip install -qr requirements.txt

In [27]:
#GPT2 runs on tf 1.x. So lets change the tf version
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


In [35]:
import os
if not os.path.exists('models/345M'):
  !python download_model.py '345M'
%ll models

total 4
drwx------ 2 root 4096 Jun  5 18:40 345M/


In [36]:
!export PYTHONIOENCODING=UTF-8

In [79]:
%cd src
import model, encoder, sample

/gdrive/My Drive/Colab_Notebooks_data/Text-completion-with-GPT-2/gpt-2/src


In [107]:
model_name='345M'
seed=0
nsamples=1
batch_size=1
length=300
temperature=1
top_k=0
models_dir='../models'

In [82]:
os.path.expanduser(os.path.expandvars(models_dir))

'../models'

In [59]:
assert nsamples % batch_size == 0

In [108]:
model_encoder = encoder.get_encoder(model_name, models_dir)

In [109]:
import json
hparams = model.default_hparams()
print(f'Default hparams:', {hparams})
with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))
print(f'Model hparams:', {hparams})

Default hparams: {HParams([('n_ctx', 1024), ('n_embd', 768), ('n_head', 12), ('n_layer', 12), ('n_vocab', 0)])}
Model hparams: {HParams([('n_ctx', 1024), ('n_embd', 1024), ('n_head', 16), ('n_layer', 24), ('n_vocab', 50257)])}


In [110]:
if length is None:
  length = hparams.n_ctx // 2
assert length < hparams.n_ctx
print(f'lenght : {length}')

lenght : 300


In [ ]:
import numpy as np
with tf.Session(graph=tf.Graph()) as session:
  context = tf.placeholder(tf.int32, [batch_size,None])
  np.random.seed(seed)
  tf.set_random_seed(seed)
  output = sample.sample_sequence(
      hparams=hparams, length=length,
      context=context,batch_size=batch_size,
      temperature=temperature, top_k=top_k)
  ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
  saver = tf.train.Saver()
  saver.restore(session, ckpt)


  while True:
    input_sent = input("Pass Text to Model >>> ")
    if input_sent:
      context_tokens = model_encoder.encode(input_sent)
      generated = 0
      for _ in range(nsamples // batch_size):
        out = session.run(output, feed_dict={
            context: [context_tokens for _ in range(batch_size)]
            })[:, len(context_tokens):]
        for i in range(batch_size):
          generated += 1
          text = model_encoder.decode(out[i])
          print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
          print(text)
      print("*" * 80)


INFO:tensorflow:Restoring parameters from ../models/345M/model.ckpt
Pass Text to Model >>> Human reason, in one sphere of its cognition, is called upon to consider questions, which it cannot decline, as they are presented by its own nature, but which it cannot answer, as they transcend every faculty of the mind.
======================================== SAMPLE 1 ========================================
 Speech is destructive of these affordances, and therefore absurd, and rhetoric to mix rationale with eloquence. There are letters, a philosopher, whose style can make up a knowledge of them better, and comprehend how manifold they are. If the acquirement of which they profess is useless, whence arises "purchase," and suppose that the only end served as a sanction for this will be to render letters an educational undertaking for those whose part it may be to apply them? what has little concernments but to find out what letter is most accessible; to discern the meaning of that word whose o